# Calculate KAM with flexible nxn sized kernel

In [57]:
#Load packages
import numpy as np
import matplotlib.pyplot as plt

from scipy.interpolate import spline
import scipy.stats
import copy
from scipy.signal import convolve2d

import matplotlib as mpl
from skimage import morphology as mph

from quat import Quat

import ebsd
import hrdic

#Set plot behaviour. interactive grain selection has only been tested in osx display mode
%matplotlib osx

## Load data

In [58]:
mapFiles = ["./example_data_AH/ebsdData"]

maps = []

for i, mapFile in enumerate(mapFiles):
    maps.append(ebsd.Map(mapFile, "cubic"))
    maps[i].loadData(mapFile, "cubic")
    maps[i].binData = maps[i].binData[::-1]  #rotate the map 180 degrees
    maps[i].buildQuatArray()
    maps[i].findBoundaries(boundDef = 6)
    maps[i].findGrains(minGrainSize=10)
    maps[i].calcGrainMisOri(calcAxis = True)

In [59]:
maps[0].plotMisOriMap()

## Not ignoring large misOris

In [76]:
def convolve(data,step):
    
    kernel = np.ones((step,step)) / (step*step)
    dataConvolved = convolve2d(data, kernel, mode='same', boundary='fill',fillvalue=0)
    
    return dataConvolved    

In [77]:
step = 3

In [78]:
quatComps = np.empty((4, maps[0].yDim, maps[0].xDim))

for i, row in enumerate(maps[0].quatArray):
    for j, quat in enumerate(row):
        quatComps[:, i, j] = quat.quatCoef
        
quatComps_convolved = np.empty((4, maps[0].yDim, maps[0].xDim))

for i in range(len(quatComps)):
    j=convolve(quatComps[i],step)
    quatComps_convolved[i]=j

kamMap = np.empty((maps[0].yDim, maps[0].xDim))
for i in range(maps[0].yDim-1):
    kamMap[i, :] = (abs(np.einsum("ij,ij->j", quatComps[:, i, :], quatComps_convolved[:, i, :])))

# kamMap[kamMap > 1] = 1

# Do the same for columns
# for i in range(maps[0].xDim-1): 
#     kamMap[:, i] += (abs(np.einsum("ij,ij->j", quatComps[:, :, i], quatComps_convolved[:, :, i])))

# kamMap /= 2

# kamMap[kamMap > 1] = 1

kamMapDegrees=np.empty((maps[0].yDim, maps[0].xDim))

# do not show values > value degkam

for i in range(maps[0].yDim):
    for j in range(maps[0].xDim):
        deg = 2 * np.arccos(kamMap[i,j]) * 180 / np.pi
        if deg > 5:
            deg = np.nan
        
        kamMapDegrees[i,j]=deg

In [79]:
plt.figure()
plt.imshow(kamMapDegrees,vmax=1)
plt.colorbar()

## Ignoring large misOris

In [60]:
def calcKamNxN(mapData,n=3,maxMisOri=5,plotKamData=False,plotGBs=False):

    quatComps = np.empty((4, mapData.yDim, mapData.xDim))
    for i, row in enumerate(mapData.quatArray):
        for j, quat in enumerate(row):
            quatComps[:, i, j] = quat.quatCoef
    data = quatComps
    
    n=n    
    if n <= 1:
        raise ValueError("Please enter a kernel dimenion n, where n > 1 and n is an odd number")
    oddCheck = n%2
    if oddCheck == 0:
        raise ValueError("Please enter a kernel dimenion n, where n > 1 and n is an odd number")
    
    else:

        r=int((n-1)/2)

        # kamData=np.empty((data.shape[1]-r,data.shape[2]-r))
        kamData=np.zeros((data.shape[1],data.shape[2])) # initialise with zeroes so that the final kamMap is the same size as the data

        for i in range(1,data.shape[1]-r): # avoid boundaries
            for j in range(1,data.shape[2]-r):

                dataKernel=np.zeros((4,n,n))
                dataCentreArray=np.zeros((4,n,n))
                misOriKernel=np.zeros((n,n))

                for q in range(data.shape[0]):

                    # define the data kernel
                    for row in range(n):
                        for column in range(n):                            
                            dataKernel[q,row,column] = data[q,i-r+row,j-r+column]

                    # define the dataCentre array
                    dataCentreArray[q,:,:] = data[q,i,j]

                # find the misOri between the central data point and the surrounding data points
                for kernelI in range(misOriKernel.shape[0]):
                    misOriKernel[kernelI,:] = abs(np.einsum("ij,ij->j", dataCentreArray[:, kernelI, :], dataKernel[:, kernelI, :])) 

                # set an upper limit for the misOri between pixels in the kernel,
                # giving values more than this limit a kernel value of 0
                kernel=np.zeros((n,n))
                for kernelI in range(kernel.shape[0]):
                    for kernelJ in range(kernel.shape[1]):
                        misoriVal=2*np.arccos(misOriKernel[kernelI,kernelJ])*180/np.pi
                        if misoriVal >= maxMisOri: # set max misOri value
                            kernalVal = 0
                        else:
                            kernalVal = 1

                        kernel[kernelI,kernelJ] = kernalVal

                # find number of zeroes in the kernel
                numZeroes=kernel.flatten().tolist().count(0)

                # convolve the kernel with the data  and get the average ori
                # find mean of data, ignoring zeroes

                convolvedDataKernel=(dataKernel*kernel)
                avOriKernel=np.zeros((4,1,1))
                for q in range(convolvedDataKernel.shape[0]):
                    meanVal=np.sum(convolvedDataKernel[q])/((n*n) - numZeroes)
                    avOriKernel[q]=meanVal        

                # find misOri of pixel wrt. averageOri
                pixelVal=dataCentreArray[:,0,0]
                pixelVal_format=np.zeros((4,1,1))
                for val in range(pixelVal_format.shape[0]):
                    pixelVal_format[val] = pixelVal[val]
                pixelVal=pixelVal_format

                misOriVal=np.zeros((1,1))

                for kernelI in range(misOriVal.shape[0]):
                    misOriVal[kernelI,:] = abs(np.einsum("ij,ij->j", avOriKernel[:, kernelI, :], pixelVal[:, kernelI, :]))
                    misOriVal=2*np.arccos(misOriVal)*180/np.pi

                # append the kam data list
                kamData[i,j] = misOriVal

        if plotKamData:
            plt.figure()
            plt.imshow(kamData,vmin=0,vmax=1)
            plt.colorbar(shrink=0.85,label="KAM [ $^o$ ]")
            
            if plotGBs:
                cmap1 = mpl.colors.LinearSegmentedColormap.from_list('my_cmap', ['black', 'black'], 256)
                cmap1._init()
                cmap1._lut[:, -1] = np.linspace(0, 1, cmap1.N + 3)

                boundaries_dilated=mph.binary_dilation(-mapData.boundaries)

                plt.imshow(boundaries_dilated, cmap=cmap1, interpolation='None', vmin=0, vmax=1)
                
        return kamData

In [80]:
kamMap3x3 = calcKamNxN(maps[0],n=3,maxMisOri=5, plotKamData=True,plotGBs=True)

/Users/mclijah5/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:49: RuntimeWarning: invalid value encountered in arccos
/Users/mclijah5/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:80: RuntimeWarning: invalid value encountered in arccos


In [81]:
kamMap5x5 = calcKamNxN(maps[0],n=5,maxMisOri=5, plotKamData=True,plotGBs=True)

/Users/mclijah5/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:49: RuntimeWarning: invalid value encountered in arccos
/Users/mclijah5/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:80: RuntimeWarning: invalid value encountered in arccos


In [82]:
kamMap7x7 = calcKamNxN(maps[0],n=7,maxMisOri=5, plotKamData=True,plotGBs=True)

/Users/mclijah5/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:49: RuntimeWarning: invalid value encountered in arccos
/Users/mclijah5/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:80: RuntimeWarning: invalid value encountered in arccos


In [83]:
kamMap9x9 = calcKamNxN(maps[0],n=9,maxMisOri=5, plotKamData=True,plotGBs=True)

/Users/mclijah5/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:49: RuntimeWarning: invalid value encountered in arccos
/Users/mclijah5/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:80: RuntimeWarning: invalid value encountered in arccos
